In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import plotly.offline as py
py.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt # side-stepping mpl backend
import matplotlib.gridspec as gridspec # subplots
import mpld3 as mpl
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from sklearn.model_selection import cross_validate
#Import models from scikit learn module:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from pandas.plotting import scatter_matrix
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

### Pre-processing

* Preprocessing is a stage that steps reduce the runtime before the model is run, eliminate contradictions in the dataset, and eliminate features that increase complexity, so the model works better. With pre-processing, the data is made available to be used in later stages.
* Also normalization is one of the pre-processing steps. In some data sets, the difference in value between data can be very large, which leads to redundancy. Normalization is used to prevent this problem. Z-transform normalization and min-max normalization are just a few of the processes used.

In [ ]:
data = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
data.head(20)

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

In [ ]:
data.drop(['Unnamed: 32',"id"], axis=1, inplace=True)
data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis]
y = data.diagnosis.values
x_data = data.drop(['diagnosis'], axis=1)


In [ ]:
sns.countplot(data['diagnosis'],label="Count")

#### Mix-max scaler
- Mix-max normalization was used to reduce or eliminate redundancy. Min-max normalization is used to scale the values of properties between 0 and 1.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x_data)
type(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

### Logistic Regression

* Logistic regression generates a probability not specific value (0 or 1).
* Logistic regression models use as their convergence criterion the maximization of a probability function. p coefficients can effortlessly converted into the corresponding odds ratios by rearing e to the coefficient if variables are represented by a single linear label and one caninterpret the magnitude of importance of a predictor.

In [ ]:
logr = LogisticRegression()
logr.fit(x_train,y_train)
y_predict_lr = logr.predict(x_test)
acc_log = metrics.accuracy_score(y_predict_lr,y_test)*100
print('LogisticRegression accuracy(in %):', acc_log)

### Support Vector Machines

* SVM is the algorithm that optimally separates between two different classes. SVM is a nonparametric algorithm and it is setting with the condition that a line is farthest from the elements of two different classes. SVM is generally used when classifying data that is linear.

In [ ]:
sv = SVC() #select the algorithm
sv.fit(x_train,y_train) # we train the algorithm with the training data and the training output
y_predict_svm = sv.predict(x_test) #now we pass the testing data to the trained algorithm
acc_svm = metrics.accuracy_score(y_predict_svm,y_test)*100
print('SVM accuracy(in %):', acc_svm)

### Decision Tree

* The DTC is an approach to multistage decision making. The main idea is any multistage approach is to split complex decisions into simpler and smaller ones up to gain the desired solution.
* Decision tree classifiers are commonly used in signal classification, image recognition, medical diagnosis, speech recognition, and more. 
* It calculates the probability of a given value to each of the categories or classifying the values to assign the most likely class.

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
y_predict_dt = dt.predict(x_test)
acc_dt = metrics.accuracy_score(y_predict_dt,y_test)*100
print('Decision Tree accuracy(in %):', acc_dt)

### Random Forest Classifier

* Random Forest classifier is selects random sub sets from data set and with that method decision trees are generated. Every decision tree makes guesses and random forest chooses most voted one. It gives grate result without any hyper parameter. It is also one of the most used algorithms because it provides simplicity and usability for both classification and regression problems.

In [ ]:
from sklearn.metrics import confusion_matrix
rf = RandomForestClassifier(n_estimators = 40)
rf.fit(x_train,y_train)
rf_pred = rf.predict(x_test)
acc_rfc = metrics.accuracy_score(rf_pred,y_test)*100
print("RandomForestClassifier accuracy(in %):", acc_rfc)

### Gaussian Naive Bayes

* Naive Bayes algorithm is one of the Machine Learning classification algorithms. It is most commonly used text classification problems. Naive Bayes is the simplest shape of Bayesian network, all attributes are unbiased given the value of the elegance variable. This is known as conditional independence. Itis apparent that the conditional independence assumption is rarely actual in maximum real-world applications

In [ ]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(x_train, y_train) 
gnb_pred = gnb.predict(x_test)
acc_gnb = metrics.accuracy_score(gnb_pred,y_test)*100
print("Gaussian Naive Bayes model accuracy(in %):", acc_gnb)

### K-Nearest Neighbours

* The k nearest neighbors algorithm (KNN) is a algorithm that used for regression and classification. KNN is a nonparametric, lazy learning algorithm. KNN is a nonparametric, lazy learning algorithm. In both cases, the input consists of the closest training examples in the feature area. KNN is an instance based learning, where the function only approaches locally and all calculations are delayed until classification.
* KNN tries to guess which group they belong to by calculating the distance of each entry from other inputs.

In [ ]:
knc = KNeighborsClassifier(n_neighbors=5)
knc.fit(x_train,y_train)
y_predict_knn = knc.predict(x_test)
acc_knn = metrics.accuracy_score(y_predict_knn,y_test)*100
print("KNN model accuracy(in %):", acc_knn)

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'Support Vector Machines','Decision Tree',
              'Random Forest Classifier','Gaussian Naive Bayes',
              'K-Nearest Neighbours'],
    'Score': [acc_log,acc_svm,acc_dt,acc_rfc,acc_gnb,acc_knn]})
models

Table of algorithm result and actual values.

In [ ]:
vecTest = y_test.reshape((-1))
result = pd.DataFrame({'ActualValues':vecTest,'LogisticRegression':y_predict_lr,'SVM':y_predict_svm,'DecisionTree':y_predict_dt,'KNN':y_predict_knn,'GausianBayes':gnb_pred,'RandomForest':rf_pred})
result

### Confusion matrix
Confusion matrix is the performance measurement of the algorithm. Confusion matrix is a 2x2 matrix and it includes 4 parameters. True Positive , False Negative , False Positive, True Negative. It can also be defined as a summary of the algorithm.
- True Positive = It is a result of the model where positive classes correctly predicted.
- True Negative = It is a result of the model where negative classes correctly predicted.
- False Positive = It is a result of the model where positive classes incorrectly predicted.
- False Negative = It is a result of the model where negative classes incorrectly predicted.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [ ]:
plt.figure(figsize=(10,15))
# Logistic Regression
plt.subplot(4,2,1)
cm = confusion_matrix(y_test, y_predict_lr) 
cm_lr = pd.DataFrame(cm)
sns.heatmap(cm_lr,cmap="Oranges",annot = True)
plt.title('Logistic Regression \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, y_predict_lr)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
#Support Machine Vector
plt.subplot(4,2,2)
cm = confusion_matrix(y_test, y_predict_svm) 
cm_svm = pd.DataFrame(cm)
sns.heatmap(cm_svm,cmap="Blues",annot = True)
plt.title('SVM \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, y_predict_svm)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
#Decision Tree
plt.subplot(4,2,3)
cm = confusion_matrix(y_test, y_predict_dt) 
cm_dt = pd.DataFrame(cm)
sns.heatmap(cm_dt,cmap="Purples",annot = True)
plt.title('Decision Tree \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, y_predict_dt)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
#KNN
plt.subplot(4,2,4)
cm = confusion_matrix(y_test, y_predict_knn) 
cm_knn = pd.DataFrame(cm)
sns.heatmap(cm_dt,cmap="Greens",annot = True)
plt.title('KNN \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, y_predict_knn)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
#Gausian Bayes
plt.subplot(4,2,5)
cm = confusion_matrix(y_test, gnb_pred) 
cm_gb = pd.DataFrame(cm)
sns.heatmap(cm_gb,cmap="Greys",annot = True)
plt.title('Gausian Bayes \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, gnb_pred)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
#Random Forest Classifier
plt.subplot(4,2,6)
cm = confusion_matrix(y_test, rf_pred) 
cm_rf = pd.DataFrame(cm)
sns.heatmap(cm_rf,cmap="Reds",annot = True)
plt.title('Random Forest \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, rf_pred)))
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.tight_layout()
plt.show()

#### Precision

- Precision shows the proportion how many of the values we estimate as Positive are truly correct.

#### Recall
- Recall shows the proportion of how many of the actual positives are correctly identified. It also called True Positive Rate. (TPR)

#### F1 Score
- F1 score is the harmonic mean of precision and recall evaluation matrices.

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [ ]:
print("Logistic Regression Precision score: {}".format(precision_score(y_test,y_predict_lr)))
print("Logistic Regression Recall score: {}".format(recall_score(y_test,y_predict_lr)))
print("Logistic Regression F1 Score: {}".format(f1_score(y_test,y_predict_lr)))

In [ ]:
print("SVM Precision score: {}".format(precision_score(y_test,y_predict_svm)))
print("SVM Recall score: {}".format(recall_score(y_test,y_predict_svm)))
print("SVM F1 Score: {}".format(f1_score(y_test,y_predict_svm)))

In [ ]:
print("Decision Tree Precision score: {}".format(precision_score(y_test,y_predict_dt)))
print("Decision Tree Recall score: {}".format(recall_score(y_test,y_predict_dt)))
print("Decision Tree F1 Score: {}".format(f1_score(y_test,y_predict_dt)))

In [ ]:
print("KNN Precision score: {}".format(precision_score(y_test,y_predict_knn)))
print("KNN Recall score: {}".format(recall_score(y_test,y_predict_knn)))
print("KNN F1 Score: {}".format(f1_score(y_test,y_predict_knn)))

In [ ]:
print("GNB Precision score: {}".format(precision_score(y_test,gnb_pred)))
print("GNB Recall score: {}".format(recall_score(y_test,gnb_pred)))
print("GNB F1 Score: {}".format(f1_score(y_test,gnb_pred)))

In [ ]:
print("Random Forest Precision score: {}".format(precision_score(y_test,rf_pred)))
print("Random Forest Recall score: {}".format(recall_score(y_test,rf_pred)))
print("Random Forest F1 Score: {}".format(f1_score(y_test,rf_pred)))

In [ ]:

plt.bar(models['Model'],models['Score'])
plt.xticks(rotation=90)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cvs_lr = cross_val_score(LogisticRegression(),x,y,cv=5)
max_lr = np.amax(cvs_lr)
cvs_svm = cross_val_score(SVC(),x,y,cv=5)
max_svm = np.amax(cvs_svm)
cvs_gnb = cross_val_score(GaussianNB(),x,y,cv=5)
max_gnb = np.amax(cvs_gnb)
cvs_rf = cross_val_score(RandomForestClassifier(n_estimators = 40),x,y,cv=5)
max_rf = np.amax(cvs_rf)

scores = pd.DataFrame({
    
    'Name' : {0:'LogisticRegression',1:'Support Vector Machines',2:'Gaussian Naive Bayes',3:'Random Forest Classifier'} ,  
    'Cross_val' : {0:cvs_lr,1:cvs_svm,2:cvs_gnb,3:cvs_rf}
    
})
scores

In [ ]:
print('Max Cross Validation Score of Logistic Regression {}'.format(max_lr))
print('Max Cross Validation Score of Support Machine Vector {}'.format(max_svm))
print('Max Cross Validation Score of Gaussian Naive Bayes {}'.format(max_gnb))
print('Max Cross Validation Score of Random Forest Classifier {}'.format(max_rf))



In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(x_data.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

### Deep Learning

Deep learning grants computational models occurring of more than one processing layers to learn the likeness of data with multiple levels of abstraction. Through these methods, technology in many other areas such as speech recognition, visual object recognition, object detection, drug discovery and genomics has been significantly improved.

###  Hyper Parameters

#### Learning Rate
* Gradient descent algorithms multiply the gradient by a scalar known as the learning rate to determine the next point.

##### Batch size
* Total number of training examples present in a single batch.

##### Epoch
* One Epoch is when an entire dataset is passed forward and backward through the neural network only once.

#### L2 Regularizer
* L2 penalizes weight2.
* The derivative of L2 is 2 * weight

#### Binary Crossentorpy 
* Binary crossentropy measures how far away from the true value. penalizes the probability based on the distance from the expected value.

##### RMSprop 
* Maintain a moving (discounted) average of the square of gradients

* Divide the gradient by the root of this average

* The RMSprop optimizer restricts the oscillations in the vertical direction.    

* Idea is to reach the global minima where the cost function attains the least possible value

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(x, y, test_size=0.20)
X_val, X_test, Y_val, Y_test = train_test_split(X_validation, Y_validation, test_size=0.40)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.regularizers import l2

In [ ]:
Y_train = Y_train.reshape((-1,1))

In [ ]:
Y_train.shape

In [ ]:
learning_rate = 0.001
epochs = 200
batch_size = 256

model = tf.keras.models.Sequential()
    
model.add(tf.keras.layers.Dense(units=8,input_shape=(X_train.shape[1],),activation= tf.nn.relu,
                                    kernel_regularizer=l2(0.001),name='Input'))
model.add(tf.keras.layers.Dense(units = 16,activation = tf.nn.relu,
                                    kernel_regularizer=l2(0.001),name='HiddenLayer-1'))
model.add(tf.keras.layers.Dense(units = 16,activation = tf.nn.relu,
                                    kernel_regularizer=l2(0.001),name='HiddenLayer-2'))
model.add(tf.keras.layers.Dense(units = 32,activation = tf.nn.relu,
                                    kernel_regularizer=l2(0.001),name='HiddenLayer-3'))
model.add(tf.keras.layers.Dense(Y_train.shape[1],activation=tf.nn.sigmoid,name='Output'))

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])

In [ ]:
history = model.fit(x=x, y=y,validation_data = (X_test,Y_test),batch_size=batch_size,epochs=epochs)

In [ ]:
model.evaluate(X_test, Y_test, verbose=1,batch_size=batch_size)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
pred = model.predict(X_test)
pred_values = pd.DataFrame(pred,columns = ['label'])
pred_values['Model_Prediction'] = (pred_values.label >0.5).astype('int')
pred_values['Actual_values'] = Y_test
pred_values.drop('label', axis = 1)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model)

In [ ]:
# sns.pairplot(data[0:10], hue='diagnosis',palette='cubehelix',kind = 'reg')

In [ ]:
clear